# Decision Tree and Bagging Classifier Example

This notebook explains a custom implementation of a Decision Tree Classifier and a Bagging Classifier. It shows how the models are built, trained, and used for predictions using a sample dataset.

In [ ]:
import numpy as np
import pandas as pd
from collections import defaultdict

## Node and Decision Tree Classifier

We define a `Node` class to represent each node in the tree and a `DecisionTreeClassifier` class that handles tree construction, splitting, and prediction.

In [56]:
class Node:
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, value=None):
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

class DecisionTreeClassifier:
    def __init__(self, max_depth=None, min_samples_split=2, feature_subset=None):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.feature_subset = feature_subset
        self.root = None

    def fit(self, X, y):
        # Convert pandas DataFrame/Series to numpy array
        if isinstance(X, pd.DataFrame) or isinstance(X, pd.Series):
            X = X.values
        if isinstance(y, pd.DataFrame) or isinstance(y, pd.Series):
            y = y.values.flatten()
        self.root = self._build_tree(X, y)

    def _build_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        n_classes = len(np.unique(y)) if n_samples > 0 else 0

        # Stopping conditions
        if (self.max_depth is not None and depth >= self.max_depth) or \
           (n_samples < self.min_samples_split) or \
           (n_classes <= 1):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        # Find best split using allowed features
        best_feature, best_threshold = self._best_split(X, y, n_features)

        if best_feature is None:
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        # Split data
        left_idxs = X[:, best_feature] <= best_threshold
        right_idxs = ~left_idxs

        left = self._build_tree(X[left_idxs], y[left_idxs], depth + 1)
        right = self._build_tree(X[right_idxs], y[right_idxs], depth + 1)

        return Node(best_feature, best_threshold, left, right)

    def _best_split(self, X, y, n_features):
        best_gini = float('inf')
        best_feature = None
        best_threshold = None

        # Use subset of features if specified
        if self.feature_subset is not None:
            features_to_consider = self.feature_subset
        else:
            features_to_consider = range(n_features)

        for feature_idx in features_to_consider:
            feature_values = X[:, feature_idx]
            unique_values = np.unique(feature_values)

            if len(unique_values) <= 1:
                continue

            thresholds = [(unique_values[i] + unique_values[i+1])/2 for i in range(len(unique_values)-1)]
            
            for threshold in thresholds:
                left_idxs = feature_values <= threshold
                right_idxs = ~left_idxs

                if np.sum(left_idxs) == 0 or np.sum(right_idxs) == 0:
                    continue

                gini = self._gini(y[left_idxs], y[right_idxs])
                if gini < best_gini:
                    best_gini = gini
                    best_feature = feature_idx
                    best_threshold = threshold

        return best_feature, best_threshold

    def _gini(self, left_y, right_y):
        n_left = len(left_y)
        n_right = len(right_y)
        n_total = n_left + n_right

        if n_total == 0:
            return 0.0

        gini_left = 1.0 - sum((np.sum(left_y == c)/n_left)**2 for c in np.unique(left_y)) if n_left !=0 else 0
        gini_right = 1.0 - sum((np.sum(right_y == c)/n_right)**2 for c in np.unique(right_y)) if n_right !=0 else 0

        return (n_left/n_total)*gini_left + (n_right/n_total)*gini_right

    def _most_common_label(self, y):
        if len(y) == 0:
            return None
        return np.argmax(np.bincount(y))

    def predict(self, X):
        if isinstance(X, pd.DataFrame) or isinstance(X, pd.Series):
            X = X.values
        return np.array([self._predict_single(x) for x in X])

    def _predict_single(self, x, node=None):
        if node is None:
            node = self.root
        if node.value is not None:
            return node.value
        if x[node.feature_index] <= node.threshold:
            return self._predict_single(x, node.left)
        else:
            return self._predict_single(x, node.right)

## Bagging Classifier

The `BaggingClassifier` class builds multiple trees using bootstrap samples of the data. It uses the out-of-bag (OOB) samples to estimate prediction error and supports random feature selection.

In [57]:
class BaggingClassifier:
    def __init__(self, n_estimators=10, max_depth=None, min_samples_split=2, max_features=None):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.max_features = max_features
        self.estimators = []
        self.oob_indices = []

    def fit(self, X, y):
        n_samples = X.shape[0]
        n_features = X.shape[1]
        self.estimators = []
        self.oob_indices = []

        for _ in range(self.n_estimators):
            # Create bootstrap sample
            bootstrap_indices = np.random.choice(n_samples, n_samples, replace=True)
            oob_mask = np.zeros(n_samples, dtype=bool)
            oob_mask[bootstrap_indices] = True
            oob_indices = np.where(~oob_mask)[0]
            self.oob_indices.append(oob_indices)

            X_boot = X.iloc[bootstrap_indices] if isinstance(X, pd.DataFrame) else X[bootstrap_indices]
            y_boot = y.iloc[bootstrap_indices] if isinstance(y, pd.Series) else y[bootstrap_indices]

            # Feature selection
            feature_subset = None
            if self.max_features is not None:
                feature_subset = np.random.choice(n_features, self.max_features, replace=False)

            # Train tree
            tree = DecisionTreeClassifier(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split,
                feature_subset=feature_subset
            )
            tree.fit(X_boot, y_boot)
            self.estimators.append(tree)

    def predict(self, X):
        if isinstance(X, pd.DataFrame):
            X = X.values
        all_preds = np.zeros((X.shape[0], self.n_estimators))
        for i, tree in enumerate(self.estimators):
            all_preds[:, i] = tree.predict(X)
        return np.array([np.argmax(np.bincount(row.astype(int))) for row in all_preds])

    def compute_oob_error(self, X, y):
        n_samples = X.shape[0]
        oob_preds = defaultdict(list)

        for i, tree in enumerate(self.estimators):
            oob_idx = self.oob_indices[i]
            if len(oob_idx) == 0:
                continue
            X_oob = X.iloc[oob_idx] if isinstance(X, pd.DataFrame) else X[oob_idx]
            preds = tree.predict(X_oob)
            for idx, pred in zip(oob_idx, preds):
                oob_preds[idx].append(pred)

        y_pred = []
        y_true = []
        for idx in range(n_samples):
            if idx in oob_preds and len(oob_preds[idx]) > 0:
                majority_vote = np.argmax(np.bincount(oob_preds[idx]))
                y_pred.append(majority_vote)
                y_true.append(y.iloc[idx] if isinstance(y, pd.Series) else y[idx])

        return np.mean(np.array(y_pred) != np.array(y_true))

## Helper Function: Print Tree

The `print_tree` function is used to visualize the decision tree structure with feature names.

In [58]:
def print_tree(node, feature_names=None, depth=0):
    if feature_names is None:
        feature_names = ['Age', 'Income', 'Student', 'Credit Rating']
    indent = "  " * depth
    if node.value is not None:
        print(f"{indent}Leaf: Class {node.value}")
        return
    print(f"{indent}Feature: {feature_names[node.feature_index]} <= {node.threshold:.2f}")
    print(f"{indent}--> True:")
    print_tree(node.left, feature_names, depth+1)
    print(f"{indent}--> False:")
    print_tree(node.right, feature_names, depth+1)

# This function recursively prints the tree structure.

## Data Loading and Preprocessing

We create a sample dataset, then encode the categorical features into numeric values for modeling.

In [59]:
data = {
    'Age': [25, 30, 35, 40, 45, 50, 55, 60],
    'Income': ['High', 'High', 'Medium', 'Low', 'Low', 'Low', 'Medium', 'High'],
    'Student': ['No', 'No', 'No', 'No', 'Yes', 'Yes', 'Yes', 'No'],
    'Credit Rating': ['Fair', 'Excellent', 'Fair', 'Fair', 'Fair', 'Excellent', 'Excellent', 'Fair'],
    'Buy Computer': ['No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No']
}
df = pd.DataFrame(data)

# Encode categorical features
df['Income'] = df['Income'].map({'Low': 0, 'Medium': 1, 'High': 2})
df['Student'] = df['Student'].map({'No': 0, 'Yes': 1})
df['Credit Rating'] = df['Credit Rating'].map({'Fair': 0, 'Excellent': 1})
df['Buy Computer'] = df['Buy Computer'].map({'No': 0, 'Yes': 1})

X = df.drop('Buy Computer', axis=1)
y = df['Buy Computer']

# This cell prepares the data for model training.

## Training the Original Decision Tree

We train a single decision tree without bagging and compute its training accuracy.

In [60]:
# Original Decision Tree
original_tree = DecisionTreeClassifier(max_depth=3)
original_tree.fit(X, y)
original_prediction = original_tree.predict(X.values)
train_accuracy = np.mean(original_prediction == y)

print("Training Accuracy of Original Decision Tree:", train_accuracy)

Training Accuracy of Original Decision Tree: 1.0


## Bagging with All Features

Next, we train a bagging ensemble using all features. We compute the out-of-bag (OOB) error as an estimate of generalization error.

In [61]:
# Bagging with all features
bagging_all = BaggingClassifier(n_estimators=10, max_depth=3, max_features=None)
bagging_all.fit(X, y)
oob_error_all = bagging_all.compute_oob_error(X, y)

print("OOB Error (All Features):", oob_error_all)

OOB Error (All Features): 0.375


## Bagging with 2 Random Features

We repeat the bagging process using only 2 randomly selected features for each tree and compute its OOB error.

In [62]:
# Bagging with 2 random features
bagging_two = BaggingClassifier(n_estimators=10, max_depth=3, max_features=2)
bagging_two.fit(X, y)
oob_error_two = bagging_two.compute_oob_error(X, y)

print("OOB Error (2 Random Features):", oob_error_two)

OOB Error (2 Random Features): 0.375


## New Data Prediction and Results

Finally, we create a new data point (Age=42, Income=Low, Student=No, Credit Rating=Excellent) and get predictions from the original tree and both bagging ensembles.

In [63]:
# New data point for prediction
new_data = pd.DataFrame([[42, 0, 0, 1]], columns=X.columns)

print("="*45)
print("Original Decision Tree (No Bagging)")
print(f"Prediction (Age=42): {'Yes' if original_tree.predict(new_data.values)[0] == 1 else 'No'}")

print("\n" + "="*45)
print("Bagging (All Features)")
print(f"OOB Error: {oob_error_all:.4f}")
print(f"Prediction (Age=42): {'Yes' if bagging_all.predict(new_data.values)[0] == 1 else 'No'}")

print("\n" + "="*45)
print("Bagging (2 Random Features)")
print(f"OOB Error: {oob_error_two:.4f}")
print(f"Prediction (Age=42): {'Yes' if bagging_two.predict(new_data.values)[0] == 1 else 'No'}")
print("="*45)

Original Decision Tree (No Bagging)
Prediction (Age=42): Yes

Bagging (All Features)
OOB Error: 0.3750
Prediction (Age=42): Yes

Bagging (2 Random Features)
OOB Error: 0.3750
Prediction (Age=42): No
